In [1]:
from suds.client import Client
import json
import pandas as pd

### DATA SOURCE: https://data.ibb.gov.tr/en/dataset/iett-hat-durak-guzergah-web-servisi

### IST Bus Stops

In [2]:
# Create a client for the WSDL
wsdl_url = "https://api.ibb.gov.tr/iett/UlasimAnaVeri/HatDurakGuzergah.asmx?wsdl"
client = Client(wsdl_url)

# Invoke the GetDurak_json operation with a blank DurakKodu parameter
json_response = client.service.GetDurak_json(DurakKodu="")

# Convert to Json object
text_string = str(json_response)
json_obj = json.loads(text_string)
print('Done!')

Done!


In [38]:
df = (
    pd.DataFrame(json_obj)
    .assign(LATITUDE=lambda x: x['KOORDINAT'].str.extract(r'POINT \(.+ (.+)\)$', expand=False).astype("float64"),
            LONGITUDE=lambda x: x['KOORDINAT'].str.extract(r'POINT \((.+) ', expand=False).astype("float64"))
    .rename(columns={"SDURAKKODU" : "STATION_CODE",
                       "SDURAKADI" : "STATION_NAME",
                       "ILCEADI" : "DISTRICT",
                       "SYON" : "DIRECTION",
                       "DURAK_TIPI" : "STOP_TYPE"})
    .drop(["KOORDINAT", "AKILLI", "FIZIKI", "ENGELLIKULLANIM"], axis=1)
)

df.head()

,STATION_CODE,STATION_NAME,DISTRICT,DIRECTION,STOP_TYPE,LATITUDE,LONGITUDE
0,225981,DENİZ HARP OKULU,Tuzla,TUZLA,CCMODERN,40.814013,29.266184
1,123932,BAŞAKPAZAR,Basaksehir,BASAKSEHIR 4. ETAP,İSTON GENİŞ,41.111396,28.810080
2,214552,ORTADAĞ,Sancaktepe,SARIGAZI,İETTBAYRAK,40.954584,29.214663
3,127912,CİHANGİR MAHALLESİ,Avcilar,B.ÇEKMECE,İETTBAYRAK,40.990910,28.713838
4,130712,ORHANGAZİ CADDESİ,Avcilar,AVCILAR-ESENYURT,İETTCAMLI,41.003121,28.709718


In [39]:
df.to_csv("tr_ist_bus_stops.csv", index=False)

### IST Bus Lines

In [2]:
# Create a client for the WSDL
wsdl_url = "https://api.ibb.gov.tr/iett/ibb/ibb.asmx?wsdl"
client = Client(wsdl_url)

# Invoke the DurakDetay_GYY operation with a blank hat_kodu parameter
response = client.service.DurakDetay_GYY(hat_kodu="")

# Print the response
print("Done!")

Done!


In [5]:
rows = response.NewDataSet.Table

In [20]:
line_df = (
    pd.DataFrame([dict(row) for row in rows])
    .rename(columns={"HATKODU" : "LINE_CODE",
                     "YON" : "DIRECTION",
                     "SIRANO" : "SEQ_NUM",
                     "DURAKKODU" : "STN_CODE",
                     "YKOORDINATI" : "LATITUDE",
                     "XKOORDINATI" : "LONGITUDE",
                     "DURAKTIPI" : "STOP_TYPE",
                     "ISLETMEBOLGE" : "BUSINESS_REGION",
                     "ISLETMEALTBOLGE" : "SUB_REGION",
                     "ILCEADI" : "DISTRICT"})
    .reindex(columns=["LINE_CODE", "DIRECTION", "SEQ_NUM", "STN_CODE", "DURAKADI", "LATITUDE", 
                      "LONGITUDE", "STOP_TYPE", "BUSINESS_REGION", "SUB_REGION", "DISTRICT"])
)

In [28]:
line_df.head()

,LINE_CODE,DIRECTION,SEQ_NUM,STN_CODE,DURAKADI,LATITUDE,LONGITUDE,STOP_TYPE,BUSINESS_REGION,SUB_REGION,DISTRICT
0,AND2Y,G,1,210141,YUNUS GARAJI,40.889562,29.206028,İETTCAMLI,Anadolu2,Pendik,Kartal
1,AND2Y,G,2,210142,YUNUS GARAJI,40.889730,29.206019,İETTBAYRAK,Anadolu2,Pendik,Kartal
2,ARN1,D,1,304561,ARNAVUTKÖY PERONLAR,41.178379,28.750063,CCMODERN,Avrupa2,Arnavutköy,Arnavutköy
3,ARN1,D,2,183682,ŞİRİNDERE CAMİİ,41.182427,28.752764,İETTCAMLI,Avrupa2,Arnavutköy,Arnavutköy
4,ARN1,D,3,107411,FATİH CADDESİ,41.180404,28.745626,CCMODERN,Avrupa2,Arnavutköy,Arnavutköy


In [26]:
for col in ["LINE_CODE", "DIRECTION", "SEQ_NUM", "STN_CODE"]:
    print(col, ":")
    print(f"Unique Categories: {len(line_df[col].unique())}")
    print(line_df[col].unique())
    print()

LINE_CODE :
Unique Categories: 1025
[AND2Y ARN1 AVR1 ... 99 99A 99Y]

DIRECTION :
Unique Categories: 2
[G D]

SEQ_NUM :
Unique Categories: 96
[1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96]

STN_CODE :
Unique Categories: 13050
[210141 210142 304561 ... 126642 183691 126751]



In [85]:
line_df.to_csv("tr_ist_bus_lines.csv", index=False)